In [3]:
## Import the required python utilities
from plotly.offline import init_notebook_mode, iplot
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import pandas as pd
import numpy as np

## Import sklearn important modules
from sklearn.decomposition import PCA, SparsePCA, MiniBatchSparsePCA, KernelPCA, IncrementalPCA
from sklearn.decomposition import TruncatedSVD, FastICA, NMF, FactorAnalysis
from sklearn.manifold import TSNE

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

init_notebook_mode(connected=True)

In [11]:
train = pd.read_csv('train.csv')
target = train['target']
train = train.drop(["target", "ID"], axis=1)

print ("Rows: " + str(train.shape[0]) + ", Columns: " + str(train.shape[1]))
train.head()

Rows: 4459, Columns: 4991


48df886f9  0deb4b6a8  34b15f335  a8cb14b00  2f0771a37  30347e683  \
0        0.0          0        0.0          0          0          0   
1        0.0          0        0.0          0          0          0   
2        0.0          0        0.0          0          0          0   
3        0.0          0        0.0          0          0          0   
4        0.0          0        0.0          0          0          0   

   d08d1fbe3  6ee66e115  20aa07010  dc5a8f1d8    ...      3ecc09859  \
0          0          0        0.0        0.0    ...            0.0   
1          0          0  2200000.0        0.0    ...            0.0   
2          0          0        0.0        0.0    ...            0.0   
3          0          0        0.0        0.0    ...            0.0   
4          0          0  2000000.0        0.0    ...            0.0   

   9281abeea  8675bec0b  3a13ed79a  f677d4d13  71b203550  137efaa80  \
0        0.0        0.0          0          0          0          0   
1        0.0        0.0          0          0          0          0   
2        0.0        0.0          0          0          0          0   
3        0.0        0.0          0          0          0          0   
4        0.0        0.0          0          0          0          0   

   fb36b89d9  7e293fbaf  9fc776466  
0          0          0          0  
1          0          0          0  
2          0          0          0  
3          0          0          0  
4          0          0          0  

[5 rows x 4991 columns]

In [5]:
feature_df = train.describe().T
feature_df = feature_df.reset_index().rename(columns = {'index' : 'columns'})
feature_df['distinct_vals'] = feature_df['columns'].apply(lambda x : len(train[x].value_counts()))
feature_df['column_var'] = feature_df['columns'].apply(lambda x : np.var(train[x]))
feature_df['target_corr'] = feature_df['columns'].apply(lambda x : np.corrcoef(target, train[x])[0][1])
feature_df.head()

columns   count          mean           std  min  25%  50%  75%  \
0  48df886f9  4459.0  14654.930101  3.893298e+05  0.0  0.0  0.0  0.0   
1  0deb4b6a8  4459.0   1390.894819  6.428302e+04  0.0  0.0  0.0  0.0   
2  34b15f335  4459.0  26722.450922  5.699652e+05  0.0  0.0  0.0  0.0   
3  a8cb14b00  4459.0   4530.163714  2.359124e+05  0.0  0.0  0.0  0.0   
4  2f0771a37  4459.0  26409.957390  1.514730e+06  0.0  0.0  0.0  0.0   

           max  distinct_vals    column_var  target_corr  
0   20000000.0             32  1.515437e+11     0.010188  
1    4000000.0              5  4.131381e+09     0.013805  
2   20000000.0             29  3.247875e+11     0.014694  
3   14800000.0              3  5.564218e+10    -0.002917  
4  100000000.0              6  2.293893e+12     0.016647

In [6]:
# variable variance
len(feature_df[feature_df['column_var'].astype(float) == 0.0]) 

256

In [12]:
feature_df = feature_df.sort_values('column_var', ascending = True)
feature_df['column_var'] = (feature_df['column_var'] - feature_df['column_var'].min()) / (feature_df['column_var'].max() - feature_df['column_var'].min())
trace1 = go.Scatter(x=feature_df['columns'], y=feature_df['column_var'], opacity=0.75, marker=dict(color="red"))
layout = dict(height=400, title='Feature Variance', legend=dict(orientation="h"));
fig = go.Figure(data=[trace1], layout=layout);
iplot(fig);

standardized_train = MinMaxScaler().fit_transform(train.values)

In [8]:
trace1 = go.Histogram(x=feature_df[feature_df['column_var'] <= 0.01]['column_var'], opacity=0.45, marker=dict(color="red"))
layout = dict(height=400, title='Distribution of Variable Variance <= 0.01', legend=dict(orientation="h"));
fig = go.Figure(data=[trace1], layout=layout);
iplot(fig);

trace1 = go.Histogram(x=feature_df[feature_df['column_var'] > 0.01]['column_var'], opacity=0.45, marker=dict(color="red"))
layout = dict(height=400, title='Distribution of Variable Variance > 0.01', legend=dict(orientation="h"));
fig = go.Figure(data=[trace1], layout=layout);
iplot(fig);

In [9]:
trace1 = go.Histogram(x=feature_df['target_corr'], opacity=0.45, marker=dict(color="green"))
layout = dict(height=400, title='Distribution of correlation with target', legend=dict(orientation="h"));
fig = go.Figure(data=[trace1], layout=layout);
iplot(fig);

In [13]:
mean_vec = np.mean(standardized_train, 0)
cov_matrix = np.cov(standardized_train.T)
eig_vals, eig_vecs = np.linalg.eig(cov_matrix)

In [14]:
# Create a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]),eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the eigenvalue, eigenvector pair from high to low
eig_pairs.sort(key = lambda x: x[0], reverse= True)

# Calculation of Explained Variance from the eigenvalues
tot = sum(eig_vals)

# Individual explained variance
var_exp = [(i/tot)*100 for i in sorted(eig_vals, reverse=True)] 
var_exp_real = [v.real for v in var_exp]

# Cumulative explained variance
cum_var_exp = np.cumsum(var_exp) 
cum_exp_real = [v.real for v in cum_var_exp]

## plot the variance and cumulative variance 
trace1 = go.Scatter(x=train.columns, y=var_exp_real, name="Individual Variance", opacity=0.75, marker=dict(color="red"))
trace2 = go.Scatter(x=train.columns, y=cum_exp_real, name="Cumulative Variance", opacity=0.75, marker=dict(color="blue"))
layout = dict(height=400, title='Variance Explained by Variables', legend=dict(orientation="h", x=0, y=1.2));
fig = go.Figure(data=[trace1, trace2], layout=layout);
iplot(fig);